In [4]:
import pandas as pd
import os
from IPython.display import FileLink

# Define paths
base_path = "/kaggle/input/bdb25-datasets/"
file_names = [f"aggregated_df_{i}.csv" for i in range(1, 10)]
file_paths = [os.path.join(base_path, file_name) for file_name in file_names]

# Combine all files
dataframes = [pd.read_csv(file_path) for file_path in file_paths]
aggregated_final_df = pd.concat(dataframes, ignore_index=True)

In [5]:
# Save the final DataFrame to a CSV file
output_path = "/kaggle/working/aggregated_final_df.csv"
aggregated_final_df.to_csv(output_path, index=False)
print(f"Aggregated file saved to {output_path}")  # File was downloaded from /kaggle/working directory

Aggregated file saved to /kaggle/working/aggregated_final_df.csv
